Notebook2 is for generating responses from the finetuned model using dataset that has been created using notebook0

### Importing dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets

### Adjusting dataset

In [ ]:
# loading dataset:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
df=pd.read_json("/content/drive/MyDrive/gemma models/Datasets/BioNLP11ID_species_dataset_validation.json")
prompts=df['gemma_prompt'][:2000]
print(f"We have {len(prompts)} rows with a valid response and others we dont have.")


We have 1344 rows with a valid response and others we dont have.


In [ ]:
print(prompts[0])


Instruction:
Do NER on this sentence."enterocolitica , and association and bioconversion of the insect in case of P"
Reply in key value pair as Tag and Entity.


### Hugging face authentication and model import
- MOdels are saved in a private repo so we need to use acess token from that repo.

In [ ]:
from huggingface_hub import notebook_login
from google.colab import userdata

!huggingface-cli login --token hf_fNGItXcvvdgrfghjkFoFboizKtQvCp # this is not correct token ask for it from MayankD


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_path =  "MayankD/bloom3b_bc5cdr_species_5epochs" # change to the path where your model is saved

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline
# Define the pipeline with max_length set to 150
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=200, device='cuda')

### Generating responses
-Also caluating time taken by each response.

In [ ]:
import json
import time

processed_texts = []

# Assume prompts is a list of prompt strings
# Assume df is a DataFrame with a 'response' column containing expected responses

for idx, prompt in enumerate(prompts):
    start_time = time.time()  # Start time of the iteration

    try:
        prompt = prompt.strip("\nInstruction:\n").strip("\\").replace("\\n", "").replace('\"', "").replace("'", "").replace("Do NER on this sentence.", "Do NER on this sentence\n")
        result = gen(f"""{prompt}""")  # Assuming gen is your function to generate response
        response = result[0]['generated_text']
        response = response[:response.find("]") + 1]
        response = response[response.find("Response:\n") + len("Response:\n"):].strip()
        processed_text = {
            'id': idx,
            "input": prompt.strip("\nInstruction:\n"),
            "expected": df.response[idx].strip("\n\nResponse:\n"),
            "response": response
        }
        processed_texts.append(processed_text)

        # Calculate elapsed time for this iteration
        elapsed_time = time.time() - start_time

        print(f"{idx + 1}/{len(prompts)} ({elapsed_time:.2f} seconds):")
        print(json.dumps(processed_text, indent=2))

        # Write to file each time in case of failure in middle of the process
        with open("/content/drive/MyDrive/bloom3b/responses_bloom3b_species_fast.json", "w") as f:
            json.dump(processed_texts, f)

    except Exception as e:
        print("Error on prompt", idx, ":", str(e))
        continue

print("Processing completed.")



1/1344 (0.68 seconds):
{
  "id": 0,
  "input": "Do NER on this sentence\nenterocolitica , and association and bioconversion of the insect in case of P\nReply in key value pair as Tag and Entity.",
  "expected": "['E-Organism:enterocolitica']",
  "response": ""
}
2/1344 (1.27 seconds):
{
  "id": 1,
  "input": "Do NER on this sentence\nGenes involved in the biosynthesis , transport and regulation of the siderophore yersiniabactin are clustered in the high pathogenicity island of Y\nReply in key value pair as Tag and Entity.",
  "expected": "[]",
  "response": "['B-Organism:Yersinia', 'E-Organism:yersiniae']"
}
3/1344 (0.76 seconds):
{
  "id": 2,
  "input": "Do NER on this sentence\nenterocolitica [ 103 ] and have counterparts in P\nReply in key value pair as Tag and Entity.",
  "expected": "['E-Organism:enterocolitica']",
  "response": "this sentence\nenterocolitica [ 103 ]"
}
4/1344 (0.76 seconds):
{
  "id": 3,
  "input": "Do NER on this sentence\nRemarkably , yersiniabactin is absent i

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Streaming output truncated to the last 5000 lines.
}
631/1344 (2.01 seconds):
{
  "id": 630,
  "input": "Do NER on this sentence\nWe have isolated several Erwinia carotovora strains , such as Ecc15 , for their capacity to persist in the Drosophila larval gut and to trigger a strong systemic immune response following oral infection [ 8 ]\nReply in key value pair as Tag and Entity.",
  "expected": "['B-Organism:Erwinia', 'E-Organism:carotovora', 'S-Organism:Ecc15', 'S-Organism:Drosophila']",
  "response": "this sentence\nWe have isolated several Erwinia carotovora strains , such as Ecc15 , for their capacity to persist in the Drosophila larval gut and to trigger a strong systemic immune response following oral infection [ 8 ]"
}
632/1344 (1.12 seconds):
{
  "id": 631,
  "input": "Do NER on this sentence\nMore recently , we isolated Pseudomonas entomophila , a Gram - negative natural bacterial pathogen of Drosophila , from flies in Guadeloupe [ 9 ]\nReply in key value pair as Tag and Enti